In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed, randrange
from scipy import stats
from functions1 import *
%matplotlib inline
seed=1



In [ ]:
x = np.arange(0, 1, 0.01)
y = np.arange(0, 1, 0.01)
#x = np.random.rand(100,1)
#y = np.random.rand(100,1)
#print(x)
#print(y)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y) +0.02*np.random.randn(100,100)
fig=plotTheSurface(x,y,z)
fig.show()
#fig.savefig('Franke.png',DPI=(200))


#OSL regression




In [ ]:
#transforming to a coluum vector in order to form an x_hat matrix
xVector=np.matrix.flatten(x)
yVector=np.matrix.flatten(y)
#vectorSize=np.size(yVector,0)
zVector=np.matrix.flatten(z)
sizeVector=np.size(zVector)
matrixSize=np.size(y,0)


In [ ]:
indeces1=np.linspace(0,sizeVector-1,sizeVector)
train1,test1=train_test_split(indeces1, 0.7)
train1=train1.astype(int)
test1=test1.astype(int)
beta,Xinv=OSLregression(xVector[train1],yVector[train1],zVector[train1],3)
zPredict=computeZpredict(xVector[test1],yVector[test1],beta,3)
print(MSE(zVector[test1],zPredict))
print(r2score(zVector[test1],zPredict))

In [ ]:
print(beta)
CI=cofidentint(Xinv,zVector[test1],zPredict,beta)
print(CI)

In [ ]:

 
# test cross validation split
indeces=np.linspace(0,sizeVector-1,sizeVector)
indeces=indeces.astype(int)
numberOfFolds=10
folds = k_folds_CV(indeces, numberOfFolds)

polynom_oders=[3,4,5]

In [ ]:
print(sizeVector)

In [ ]:
zPredictmatrix,statsMatrix,betaMatrix=olsModel(polynom_oders,xVector,yVector,zVector,numberOfFolds,folds,indeces)


In [ ]:
bias,var=computeBiasandVar(zPredictmatrix[:,:,2],zVector)
print('Bias^2 for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))
beta1=np.mean(betaMatrix[:,:,2],1)
z_pred=computeZpredict(xVector,yVector,beta1,5)
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(z_pred,(matrixSize,matrixSize))
fig1=plotTheSurface(x,y,zPredictReshape)
fig1.show()
#fig1.savefig('olsFranke.png')
print(str(MSE(z_pred,zVector)))
print(bias + var)

In [ ]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.0001,normalize=True)
lassoreg.fit(xMatrix,zVector)
z_pred = lassoreg.predict(xMatrix)

In [ ]:
from sklearn.linear_model import Ridge
xMatrix=constructX(xVector,yVector,3)
ridgereg = Ridge(alpha=0.001,fit_intercept=True)
ridgereg.fit(xMatrix,zVector)
z_pred = ridgereg.predict(xMatrix)
print(MSE(zVector,z_pred))
print(r2score(zVector,z_pred))
print(ridgereg.coef_)
print(ridgereg.intercept_)

In [ ]:
beta,xx=Ridgeregression(xVector,yVector,zVector,3,0.001)
zPredictmatrix=computeZpredict(xVector,yVector,beta,3)
MSE1=MSE(zVector,zPredictmatrix)
r2score1=r2score(zVector,zPredictmatrix)
print(MSE1)
print(r2score1)
print(beta)

In [ ]:
#printing results
lambda_values=[1e-3,1e-2,1e-1,1,10,1e2]
zPredictmatrix,statsMatrix,betaMatrix=ridge_regress(lambda_values,polynom_oders,xVector,yVector,zVector,numberOfFolds,folds,indeces)
for j,order in enumerate(polynom_oders):
    for  h,lbd in enumerate(lambda_values): 
        print(lbd)
        print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,h,j])))
        print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,h,j])))
        print('\n')

beta1=np.mean(betaMatrix[:,:,0,2],1)
z_pred=computeZpredict(xVector,yVector,beta1,5)
bias,var=computeBiasandVar(zPredictmatrix[:,:,0,2],zVector)
print('Bias for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))

matrixSize=np.size(y,0)
zPredictReshape=np.reshape(z_pred,(matrixSize,matrixSize))
#fig2=plotTheSurface(x,y,zPredictReshape)
#fig2.show()
#fig2.savefig('RidgeFranke.png',DPI=(200))  
print(str(MSE(z_pred,zVector)))
print(bias + var)

In [ ]:
print(betaMatrix[:,:,0,2])


In [ ]:
lambda_values=[1e-3,1e-2,1e-1,1,10,1e2]
zPredictmatrix,statsMatrix,betaMatrix=lassoRegress(lambda_values,polynom_oders,xVector,yVector,zVector,numberOfFolds,folds,indeces)
for j,order in enumerate(polynom_oders):
    for  h,lbd in enumerate(lambda_values): 
        print(lbd)
        print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,h,j])))
        print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,h,j])))
        print('\n')

In [ ]:
beta1=np.mean(betaMatrix[:,:,0,2],1)
z_pred=computeZpredict(xVector,yVector,beta1,5)
bias,var=computeBiasandVar(zPredictmatrix[:,:,0,2],zVector)
print('Bias for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(z_pred,(matrixSize,matrixSize))
#fig2=plotTheSurface(x,y,zPredictReshape)
#fig2.show()
#fig2.savefig('LassoFranke.png')
print(str(MSE(z_pred,zVector)))
print(bias + var)

In [ ]:
print(betaMatrix[:,:,0,2])